# TODO

In [1]:
# import wmfdata as wmf
import wmfdata as wmf
from wmfdata import mariadb, hive, spark
from wmfdata.utils import pct_str, pd_display_all

import requests
import re
from urllib import request
import json

import logging
import gc
import weakref
from functools import reduce
from pathlib import Path
import requests
import pprint
import os
import warnings

#import time
#import datetime dt
#from datetime import datetime, timedelta, date
#import dateutil

from pathlib import Path
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
#%load_ext sql_magic

In [2]:
#MONTHLY -------------------------------------------

#Decision log
#https://docs.google.com/document/d/1GLMPIuDmA6_BY56SeNab66jGAEB_1YuRJ2FWSdJfYTA/edit#heading=h.ivyvml86ev1
    
#initial short list of wikis deployed to #https://meta.wikimedia.org/wiki/List_of_Wikipedias
wikis = ['wikishared']#metawiki
wikis_l = pd.DataFrame(wikis)
wikis_l = wikis_l.rename(columns={0: 'database_code'})
wikis_df_list = wikis_l['database_code'].tolist()
wikis_df_tuple = tuple(wikis_df_list)

last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)
METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")
metrics_month = pd.Period(METRICS_MONTH_TEXT)

#fiscal_quarter = pd.PeriodIndex(metrics_month, freq='Q-JUL').strftime('Qtr%q')

query_vars = dict(
    wikis_to_query = wikis_df_tuple,
    snapshot = MEDIAWIKI_HISTORY_SNAPSHOT,
    metrics_month= str(metrics_month),
    metrics_month_start= str(metrics_month.start_time), 
    metrics_month_first_day= str(metrics_month.asfreq("D", how="start")),
    metrics_month_end= str((metrics_month + 1).start_time),
    
    year = metrics_month.asfreq("D", how="start").strftime("%Y"),
    month = metrics_month.asfreq("D", how="start").strftime("%m"),
    month_int = pd.to_numeric(metrics_month.asfreq("D", how="start").strftime("%m")),
    month_int_end = pd.to_numeric((metrics_month +1).asfreq("D", how="start").strftime("%m")),
    
    utc_start = metrics_month.asfreq("D", how="start").strftime("%Y%m%d%H%M%S"),
    utc_end = (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d%H%M%S"),
    day_before_utc_end = ((metrics_month-1)).asfreq("D", how="start").strftime("%Y%m%d%H%M%S"),
    
    metrics_month_last_day= str(metrics_month.asfreq("D", how="end")),
    api_metrics_month_first_day= metrics_month.asfreq("D", how="start").strftime("%Y%m%d"),
    api_metrics_month_next_month_first_day= (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    api_metrics_month_day_after= (metrics_month + 1).asfreq("D", how="start").strftime("%Y%m%d"),
    
    metrics_prev_month= str(metrics_month - 1),
    retention_cohort= str(metrics_month - 2),
)

In [3]:
%store query_vars

Stored 'query_vars' (dict)


# Query

In [8]:
#       campaign_events.event_status, --Status of the event ("open", "closed"...)
#       campaign_events.event_type, -- options: generic, 
#       campaign_events.event_meeting_url, --For online events, the URL of the video call
# ce_event_address.ceea_id,--- in person address?

events_query = '''
    SELECT 
       campaign_events.event_id, 
       campaign_events.event_name, 
       campaign_events.event_page_title,
       campaign_events.event_page_wiki AS database_code, 
       campaign_events.event_chat_url,
       campaign_events.event_status, 
       campaign_events.event_start_utc, 
       campaign_events.event_end_utc, 
       campaign_events.event_type, 
       campaign_events.event_meeting_type, 
       campaign_events.event_meeting_url, 
       campaign_events.event_created_at,
       campaign_events.event_deleted_at,
       campaign_events.event_last_edit,
       ce_event_address.ceea_id,
       ce_address.cea_country,
       ce_tracking_tools.cett_tool_id, 
       ce_tracking_tools.cett_tool_event_id
    FROM campaign_events 
    LEFT JOIN ce_event_address
        ON campaign_events.event_id = ce_event_address.ceea_event
    LEFT JOIN ce_address 
        ON ce_address.cea_id = ce_event_address.ceea_address
    LEFT JOIN ce_tracking_tools
        ON campaign_events.event_id = ce_tracking_tools.cett_event
    WHERE 
        campaign_events.event_created_at BETWEEN "{utc_start}" and "{utc_end}" OR
        campaign_events.event_start_utc BETWEEN "{utc_start}" and "{utc_end}" OR
        campaign_events.event_last_edit BETWEEN "{utc_start}" and "{utc_end}"
    
    '''

campaign_events = mariadb.run(events_query.format(**query_vars), 'wikishared')


/home/iflorez/.conda/envs/2023-09-14T16.13.36_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


## Query for pageviews

In [ ]:
# Filter out specific warning messages by category or message
warnings.filterwarnings("ignore", category=Warning)

In [5]:
query_vars['pv_page_title'] = tuple(('Event:' + campaign_events['event_page_title'].astype(str)).unique().tolist())
#query_vars['database_code'] = tuple(campaign_events['database_code'].unique().tolist())

In [6]:
#TODO Consider: do we want pageviews during the contest period only?
# replace project and database code so that it works with more than one wiki
# domain_name.str.rstrip('(.org)') 

event_pageviews = '''
SELECT  
   sum(view_count) AS pageviews, 
   page_title AS pv_page_title
FROM wmf.pageview_hourly  
WHERE 
  year       = {year}            AND
  month      >= {month_int}      AND
  month      < {month_int_end}   AND
  agent_type = 'user'            AND
  project = 'meta.wikimedia'     AND
  page_title IN {pv_page_title}
  --page_id  = 	12164639
GROUP BY page_title
    '''

campaign_pv = spark.run(event_pageviews.format(**query_vars))
#remove prefix
campaign_pv['page_title'] = campaign_pv['pv_page_title'].map(lambda x: x.lstrip('Event:'));

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 20:35:41 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/09/12 20:35:42 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/09/12 20:35:42 WARN Utils: Service 'sparkDriver' could not bind on port 12001. Attempting port 12002.
23/09/12 20:35:42 WARN Utils: Service 'sparkDriver' could not bind on port 12002. Attempting port 12003.
23/09/12 20:35:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/12 20:35:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/09/12 20:35:42 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/09/12 20:35:52 WARN Utils: Service 'org.apache.spark.netwo

### Address test rows

In [7]:
#Remove rows called test, myFirstEvent, myfirstevent, testevent -- in any language
string_test_remove = pd.read_csv('test_in_languages.csv')
remove = string_test_remove['test'].unique().tolist()
additional_strings = ['My_fun_event','The_Lion_King!','winhisintlpol_test','example', 'testezzefzefzef']

remove_substrings_list = remove + additional_strings

In [8]:
#categorize instead of removing
for name in remove_substrings_list:
     campaign_events.loc[campaign_events.event_name.str.contains(name), 'event_status'] = 5 # 5 = 'test'

# Get participants count

# TODO potentially redo this query away from for loop as with the preceeding ones

In [9]:
ce_participants_count = []

def get_participants_count_mariadb(df):
    participants_query = '''
    SELECT 
       campaign_events.event_id,
       COUNT(ce_participants.cep_user_id) AS participants_register_count,
       SUM(CASE WHEN cep_unregistered_at IS NULL THEN 0  ELSE 1 END) AS participant_unregister_count
    FROM ce_participants
    JOIN campaign_events
    ON cep_event_id = campaign_events.event_id
    WHERE campaign_events.event_id = {event_id}
    '''
    for event_id in df['event_id'].unique():
        participants_r = mariadb.run(participants_query.format(event_id=event_id), 'wikishared')
        ce_participants_count.append(participants_r)   
    
    return(ce_participants_count)




#campaign_events_participants = mariadb.run(participants, 'wikishared')

In [10]:
#run function
participants = get_participants_count_mariadb(campaign_events)

#object list to df, selecting only one item '_item_cache'
participants_to_df = pd.DataFrame([vars(d) for d in participants])[['_item_cache']].apply(pd.Series)

#explode dictionary to columns
p_df = participants_to_df['_item_cache'].apply(pd.Series)


#clean up object columns so they have only the integer values
p_df['event_id'] = p_df['event_id'].astype(str).replace(r'\nName: event_id, dtype: int64','', regex=True).replace('0    ','', regex=True).astype(int)
p_df['participants_register_count'] = p_df['participants_register_count'].astype(str).replace(r'\nName: participants_register_count, dtype: int64','', regex=True).replace('0    ','', regex=True).astype(int)
p_df['participant_unregister_count'] = p_df['participant_unregister_count'].astype(str).replace(r'None\nName: participant_unregister_count, dtype: object','', regex=True).replace('0    ','', regex=True).replace(r'\nName: participant_unregister_count, dtype: float64',0, regex=True).astype(str).replace('0    ','', regex=True).apply(pd.to_numeric, errors='coerce').fillna(0.0)
p_df['participants'] = p_df['participants_register_count'] - p_df['participant_unregister_count']

#0.0\nName: participant_total_unregisters, dtype: float64

/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/iflorez/.conda/envs/2023-09-11T19.34.12_ifl

In [11]:
ce_organizers_count = []

def get_organizers_count_mariadb(df):
    organizers_query = '''
    SELECT 
       campaign_events.event_id,
       COUNT(ce_organizers.ceo_user_id) AS organizers_count
    FROM ce_organizers
    JOIN campaign_events
    ON ceo_event_id = campaign_events.event_id
    WHERE campaign_events.event_id = {event_id}
    '''
    for event_id in df['event_id'].unique():
        organizers_r = mariadb.run(organizers_query.format(event_id=event_id), 'wikishared')
        ce_organizers_count.append(organizers_r)   
    
    return(ce_organizers_count)


In [12]:
#run function
organizers = get_organizers_count_mariadb(campaign_events)

#object list to df, selecting only one item '_item_cache'
organizers_to_df = pd.DataFrame([vars(d) for d in organizers])[['_item_cache']].apply(pd.Series)

#explode dictionary to columns
o_df = organizers_to_df['_item_cache'].apply(pd.Series)
#clean up object columns so they have only the integer values
o_df['event_id'] = o_df['event_id'].astype(str).replace(r'\nName: event_id, dtype: int64','', regex=True).replace('0    ','', regex=True).astype(int)
o_df['organizers'] =o_df['organizers_count'].astype(str).replace(r'\nName: organizers_count, dtype: int64','', regex=True).replace('0    ','', regex=True).astype(int)

/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/iflorez/.conda/envs/2023-09-11T19.34.12_ifl

In [13]:
event_data = campaign_events.merge(p_df, how='left', on='event_id').merge(o_df, how='left', on='event_id').merge(campaign_pv, how='left', left_on='event_page_title',right_on='page_title');
                                                                                                                

### clean event chat url 

In [14]:
event_data['event_chat_url_clean'] = event_data['event_chat_url'].replace({'https://chat.whatsapp.com': 'whatsapp',
                                                                           'https://wikimedia.cm/': 'wikimedia',
                                                                           'https://testgroupchaturl.org':'test',
                                                                           'https://wikimedia.cm/':'wikimedia.cm',
                                                                           'https://meet.google.com':'google meet',
                                                                           'https://t.me/': 'Telegram',
                                                                           'https://meta.wikimedia.org/wiki/Event_talk': 'event talk page',
                                                                           'https://docs.google.com/forms': 'google forms',
                                                                           'https://docs.google.com': 'google docs'
                                }, regex=True).str.split('/').str[0]#.replace(r"([^\/]+$)","")#replace all after slash

# Query for Participant usernames

In [15]:
#get event ids
query_vars['cep_event_id_tuple'] = tuple(event_data['event_id'].unique().tolist())

In [16]:
    participant_dig = '''
    SELECT 
       cep_event_id  AS event_id,
       cep_user_id AS user_id,
       cep_registered_at,
       cep_unregistered_at
    FROM ce_participants
    WHERE cep_event_id IN {cep_event_id_tuple}

    '''

participant_dig_data = mariadb.run(participant_dig.format(**query_vars), 'wikishared')

/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [17]:
#get participant ids
query_vars['cep_user_id_tuple'] = tuple(participant_dig_data['user_id'].unique().tolist())

In [18]:
#GET usernames
#fyi, this table can also provide gu_home_db AS home_db
user_names_participants_query =  '''
SELECT gu_name AS username,
gu_id AS user_id
FROM globaluser 
WHERE globaluser.gu_id IN {cep_user_id_tuple}
'''

user_names_p = mariadb.run(user_names_participants_query.format(**query_vars), 'centralauth')

/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Query for Organizer data

In [19]:
    organizer_dig = '''
    SELECT 
       ceo_event_id AS event_id,
       ceo_user_id  AS user_id
    FROM ce_organizers
    WHERE ceo_event_id IN {cep_event_id_tuple}

    '''

organizer_dig_data = mariadb.run(organizer_dig.format(**query_vars), 'wikishared')

/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [20]:
#get organizer ids
query_vars['ceo_user_id_tuple'] = tuple(organizer_dig_data['user_id'].unique().tolist())

In [21]:
#GET usernames
#gu_home_db AS home_db,
#gu_registration

user_names_organizer_query =  '''
SELECT gu_name AS username,
gu_id AS user_id
FROM globaluser 
WHERE globaluser.gu_id IN {ceo_user_id_tuple}
'''

user_names_organizers = mariadb.run(user_names_organizer_query.format(**query_vars), 'centralauth')

/home/iflorez/.conda/envs/2023-09-11T19.34.12_iflorez/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


### combine

In [22]:
participant_df = participant_dig_data.merge(user_names_p, how='left', on='user_id')
participant_df['editor_type'] = 'event_participant'

In [23]:
organizer_df = organizer_dig_data.merge(user_names_organizers, how='left', on='user_id')
organizer_df['editor_type'] = 'event_organizer'

In [24]:
editors = pd.concat([participant_df, organizer_df])

# Query for editor data

In [25]:
#get 
query_vars['usernames'] = tuple(editors['username'].unique().tolist())
query_vars['user_ids'] = tuple(editors['user_id'].unique().tolist())

In [26]:
#New Campaign User - https://phabricator.wikimedia.org/T329382
#a) this is the first logging of their username (a global account)
#b) it is not auto logged
#c) i) returnTo field = campaign page and the campaign is happening or going to happen in the next 30 days 
#OR
#c) ii) account created within 30 days of their signing up for an event 

#LPAD auto coerces number to string on Spark
#event.SSAC has 90 days of data, but event_sanitized.SSAC will have longer time data for columns in allowlist. 
# FROM event.serversideaccountcreation
#--event.returnTo AS acp,


#This gives rows that are between x and y months prior. 
acp_ssac = spark.run("""
SELECT event.userName AS username,
    CONCAT(cast(year as string), '-', LPAD(cast(month as string), 2, '0'), '-', LPAD(day, 2, '0')) AS `ac_date`
FROM event_sanitized.serversideaccountcreation  
WHERE 
    event.userName IN {usernames} AND
    event.isselfmade = true AND
    CONCAT(cast(year as string), '-', LPAD(cast(month as string), 2, '0'), '-', LPAD(day, 2, '0')) >= date_sub(current_date(), 90)
""".format(**query_vars))

In [27]:
#TODO this data could also be pulled above for participants and organizers FROM globaluser...if we can also pull NOT auto_created
acp_ssac['ac_date'] = pd.to_datetime(acp_ssac['ac_date'], format = "%Y-%m-%d")#.dt.strftime("%Y%m%d")
editors['cep_registered_at'] = pd.to_datetime(editors['cep_registered_at'], format = "%Y%m%d%H%M%S")#.dt.strftime("%Y%m%d")


potentially_new_people = (acp_ssac.merge(editors[['user_id', 'event_id', 'cep_registered_at','username']], how='left', on='username')
                                 .merge(event_data[['event_id', 'event_created_at', 'event_start_utc','event_end_utc']], how='left', on='event_id')
                         )

#time delta (days) between account_creation date and 'cep_registered_at'
potentially_new_people['ac_reg_delta']=(potentially_new_people.cep_registered_at-potentially_new_people.ac_date).dt.days

In [28]:
#keep only rows of new folks that joined an event within 30 days of registering
new_people = potentially_new_people[potentially_new_people['ac_reg_delta']<=30]

#keep only their first registration (sort by name and sort by their registration timestamps)
new_people_selects_prep = new_people[['username','ac_date','ac_reg_delta','cep_registered_at','event_created_at','event_id']]
new_people_selects_prep.sort_values(by=["username",'cep_registered_at']).drop_duplicates('username')

#Remove duplicates by columns username, keeping the row with the lowest value in column ac_reg_delta
new_people_selects = new_people_selects_prep.groupby('username', group_keys=False).apply(lambda x: x.loc[x.ac_reg_delta.idxmin()])
new_people_selects.reset_index(drop = True, inplace = True)

In [89]:
new_people_selects['username'].duplicated().any()

#check for duplicates that were missed
#pd.concat(g for _, g in new_people_selects.groupby("username") if len(g) > 1)

False

In [90]:
#create a new_df with the above showing only the count of new people per each event
new_people_by_event = new_people_selects.groupby('event_id')['username'].nunique().reset_index().rename(columns={'username':'new_users_count'})

In [30]:
#COUNT(DISTINCT em.user_id) AS total_accounts,
#MIN(em.user_registration) AS em_registration,
    
#SUM(em.edits) AS total_edits, 
#SUM(em.content_edits) AS total_content_edits, 
#SUM(em.visual_edits) AS total_visual_edits, 
#SUM(CAST(em.mobile_app_edits AS INT))+SUM(CAST(em.mobile_web_edits AS INT)) AS mobile_edits,

#MAX(event_timestamp) AS last_activity_date,
#MIN(event_user_first_edit_timestamp) AS first_activity_date,

In [36]:
#https://wikitech.wikimedia.org/wiki/Analytics/Data_Lake/Edits/MediaWiki_history
#https://github.com/IreneFlorez/GLOW/blob/master/scripts/data_wrangling/8a_collect_contest_editors_region.ipynb
#https://github.com/wikimedia/analytics-refinery/blob/master/oozie/edit/hourly/edit_hourly.hql -- for user_tenure_bucket and user_edit_count_buckets
#https://gerrit.wikimedia.org/r/c/analytics/refinery/+/889766/2/hql/edit/edit_hourly.hql --latest update
#       --MAX(event_timestamp) AS last_activity_date,
editor_data_query_results_em = spark.run("""

WITH editor_month AS (
SELECT em.user_name AS username,
    COUNT(DISTINCT em.wiki) AS total_wikis, 
    SUM(CAST(em.bot_by_group AS INT)) AS bot_history,
    COUNT(DISTINCT CASE WHEN em.wiki LIKE 'commonswiki' THEN  em.wiki END) AS commons_editor,
    COUNT(DISTINCT CASE WHEN em.wiki LIKE 'wikidatawiki' THEN  em.wiki END) AS wikidata_editor,
    CASE
        WHEN SUM(em.edits) < 5 THEN '1-4'
        WHEN SUM(em.edits) >= 5 AND SUM(em.edits) < 100 THEN '5-99'
        WHEN SUM(em.edits) >= 100 AND SUM(em.edits) < 1000 THEN '100-999'
        WHEN SUM(em.edits) >= 1000 AND SUM(em.edits) < 10000 THEN '1000-9999'
        WHEN SUM(em.edits) >= 10000 THEN '10000+'
        ELSE 'Undefined'
    END AS user_edit_count_bucket
from wmf_product.editor_month em
WHERE user_name IN {usernames} 
GROUP BY user_name
),

mwh AS (
SELECT DISTINCT(event_user_text)  AS username,
       MAX(event_timestamp) OVER (PARTITION BY event_user_text)  AS last_activity_date,
       --MIN(event_user_first_edit_timestamp) AS first_activity_date,
       MAX(UNIX_TIMESTAMP(event_timestamp, 'yyyy-MM-dd HH:mm:ss.SSS') - UNIX_TIMESTAMP(
            LEAST(
                COALESCE(event_user_registration_timestamp, event_user_creation_timestamp, event_user_first_edit_timestamp),
                COALESCE(event_user_creation_timestamp, event_user_first_edit_timestamp, event_user_registration_timestamp),
                COALESCE(event_user_first_edit_timestamp, event_user_registration_timestamp, event_user_creation_timestamp)
            ),'yyyy-MM-dd HH:mm:ss.SSS')) OVER (PARTITION BY event_user_text)  AS user_tenure
FROM wmf.mediawiki_history 
WHERE  
    event_entity = 'revision'          AND
    event_type = 'create'              AND
    event_user_text IN {usernames}    AND
    snapshot == '{snapshot}'        AND
    event_timestamp IS NOT NULL        
)


SELECT
    mwh.username, 
    last_activity_date,
    CASE
        WHEN user_tenure < 86400 THEN 'Under 1 day'
        WHEN user_tenure >= 86400 AND user_tenure < 7*86400 THEN '1 to 7 days'
        WHEN user_tenure >= 7*86400 AND user_tenure < 30*86400 THEN '7 to 30 days'
        WHEN user_tenure >= 30*86400 AND user_tenure < 90*86400 THEN '30 to 90 days'
        WHEN user_tenure >= 90*86400 AND user_tenure < 365*86400 THEN '90 days to 1 year'
        WHEN user_tenure >= 365*86400 AND user_tenure < 1095*86400 THEN '1 to 3 years'
        WHEN user_tenure >= 1095*86400 AND user_tenure < 3650*86400 THEN '3 to 10 years'
        WHEN user_tenure >= 3650*86400 THEN 'Over 10 years'
        ELSE 'Undefined'
    END AS user_tenure_bucket, --Bucketed time between user creation and edit (Under 1 day, 1 to 7 days, 7 to 30 days, ..., Over 10 years, Undefined)
    total_wikis,
    bot_history,
    commons_editor,
    wikidata_editor
FROM editor_month
LEFT JOIN mwh 
ON editor_month.username = mwh.username

""".format(**query_vars))

# Add in countries

In [40]:
query_vars['countries'] = tuple(event_data['cea_country'].dropna().unique().tolist())

In [41]:
countries_df = spark.run("""
SELECT
    r.wmf_region,
    r.canonical_country_name AS cea_country
FROM gdi.country_meta_data AS r
WHERE 
    r.canonical_country_name IN {countries}
""".format(**query_vars))

# Outputs

## Event

In [91]:
organizers_only_from_editor_data = editors[editors['editor_type']=='event_organizer'][['event_id','username']]
organizers_only_from_editor_data = organizers_only_from_editor_data.rename(columns={'username':'organizer_username'}) 

#count of organizers per event
#organizers_per_event = organizers_only_from_editor_data.groupby('event_id')['organizer_username'].nunique().reset_index().rename(columns={'organizer_username':'organizer_count'})

#unique orgnizers per event to a list
orgnizers_list_per_event = organizers_only_from_editor_data.groupby('event_id')['organizer_username'].unique().reset_index()

In [92]:
event_roster = (event_data.merge(orgnizers_list_per_event, how='left', on='event_id')
                          .merge(countries_df, how='left', on='cea_country')
               )

#merge(organizers_per_event, how='left', on='event_id')
                        

In [93]:
subset_selection = ['participants_register_count', 'participant_unregister_count','participants','event_id', 'event_name', 'event_page_title', 'database_code','event_chat_url','event_status', 'event_start_utc', 'event_end_utc', 'event_type', 'event_meeting_type','event_meeting_url','event_created_at', 'event_deleted_at', 'event_last_edit','ceea_id', 'cea_country','cett_tool_id', 'cett_tool_event_id','pageviews', 'pv_page_title', 'page_title', 'event_chat_url_clean', 'wmf_region', 'new_users_count']

#list that can't be used in drop_duplicates:
#'organizers_count', 'organizer_username
                                
base_view = (event_roster.merge(new_people_by_event, how='left', on='event_id')
             .drop_duplicates(subset=subset_selection)
             )

In [94]:
#reorder columns
base_view = base_view[['event_id',
                       'event_page_title',
                       'cea_country','wmf_region', 
                       'pageviews',
                       'organizers','organizer_username',
                       'new_users_count',
                       'participants',
                       'event_status',
                       'event_created_at','event_start_utc', 'event_end_utc', 'event_last_edit','event_deleted_at',
                       'event_chat_url_clean','cett_tool_id','cett_tool_event_id','event_meeting_url',
                       'participants_register_count','participant_unregister_count',
                       'event_type','event_meeting_type'
                      ]].rename(columns={'cea_country': 'country',
                                        #'participant_total_registers':'participants'
                                        })


In [95]:
base_view.to_csv('output/wrangling/base_view.csv', index=False)

## Editors

In [96]:
#merge 
editor_data = (editors.merge(editor_data_query_results_em, 
                             how='left', 
                             on='username'
                            )
                      .merge(acp_ssac, 
                             how='left', 
                             on='username'
                            )
                      .merge(base_view[['event_id',
                                        'country',
                                        'wmf_region',
                                        'event_status'
                                       ]],
                             how='left', 
                             on=['event_id']
                            )
                      .merge(new_people_selects[['username', 'ac_reg_delta'
                                                ]],
                             how='left', 
                             on='username'
                            )
                      
              )               

In [99]:
editor_data.to_csv('output/wrangling/editors.csv', index=False)